In [ ]:
"""
Tracing using InterpolatedBoozerField (The most common use case)
"""
"""
SOME BENCHMARKS FOR REFERENCE

M1 Macbook Pro
==============
res 48 interpolation ~60s
env OMP_NUM_THREADS=1 mpiexec -n 8 python tracing_interp_example.py

res 96 interpolation ~650s
env OMP_NUM_THREADS=2 mpiexec -n 4 python tracing_interp_example.py

res 96 interpolation ~900s
mpiexec -n 2 python tracing_interp_example.py


Ginsburg HPC
==============
res 48 interpolation ~20s
-N 1, --ntasks-per-node=32, --cpus-per-task=1

res 48 interpolation ~35s
-N 1, --ntasks-per-node=8, --cpus-per-task=4

res 96 interpolation ~280s
-N 1, --ntasks-per-node=8, --cpus-per-task=4

"""
from simsopt.field.boozermagneticfield import BoozerRadialInterpolant, InterpolatedBoozerField
from simsopt.field.tracing import trace_particles_boozer, MinToroidalFluxStoppingCriterion, MaxToroidalFluxStoppingCriterion
from simsopt.util.constants import ALPHA_PARTICLE_MASS, ALPHA_PARTICLE_CHARGE, FUSION_ALPHA_PARTICLE_ENERGY
import os
import sys
import numpy as np
from simsopt.util.constants import PROTON_MASS, ELEMENTARY_CHARGE, ONE_EV
from booz_xform import Booz_xform
import matplotlib.pyplot as plt
import time

try:
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
except ImportError:
    comm = None

resolution = 16 # 96 used for production runs

reltol = 1e-12 # relative tolerance for integration
abstol = 1e-12 # absolute tolerance for integration
boozmn_filename = './initial conditions/simsopt initial/boozmn_LandremanPaul2021_QA_reactorScale_lowres_reference.nc' # netcdf file, 4608 modes in total
tmax = 1e-3 # Time for integration
ns_interp = resolution # Number of radial grid points for interpolation
ntheta_interp = resolution # Number of poloidal angle grid points for interpolation
nzeta_interp = resolution # Number of toroidal angle grid points for interpolation
N = -4

## Setup Booz_xform object
equil = Booz_xform()
equil.verbose = 0
equil.read_boozmn(boozmn_filename)
nfp = equil.nfp

## Call boozxform and setup radial interpolation of magnetic field
order = 1
bri = BoozerRadialInterpolant(equil, order, no_K=True, comm=comm) # if equil is Vmec object, mpi will be obtained from equil directly

## Setup 3d interpolation of magnetic field
"""
It takes time to set up InterpolatedBoozerField, but future field
evalutions will be much faster comparing to using BoozerRadialInterpolant.

It is important to make sure that different mpi processes are not out of sync
and stuck while waiting for data communication.
It might be easier to set up all the quantities you might need using
the initialize parameter.

You can find what quantities are needed for each RHS class in tracing.cpp.

"""
degree = 3
srange = (0, 1, ns_interp)
thetarange = (0, np.pi, ntheta_interp)
zetarange = (0, 2*np.pi/nfp, nzeta_interp)
initialize = ["psip", "G", "I", "dGds", "dIds", "iota", "modB_derivs", "modB"]
t = time.time()
field = InterpolatedBoozerField(bri, degree, srange, thetarange, zetarange, True, nfp=nfp, stellsym=True, initialize=initialize)

Ekin=FUSION_ALPHA_PARTICLE_ENERGY
mass=ALPHA_PARTICLE_MASS
charge=ALPHA_PARTICLE_CHARGE # Alpha particle charge
vpar0=np.sqrt(2*Ekin/mass)

# mu0_vals    = np.loadtxt('mu0.txt')      # Magnetic moment initial values, if needed later
s0_vals     = np.loadtxt('./initial conditions/simsopt initial/s0.txt')       # s-coordinate initial values
theta0_vals = np.loadtxt('./initial conditions/simsopt initial/theta0.txt')   # θ-coordinate initial values
vpar0_vals  = np.loadtxt('./initial conditions/simsopt initial/vpar0.txt')    # v_parallel initial values
zeta0_vals  = np.loadtxt('./initial conditions/simsopt initial/zeta0.txt')    # ζ-coordinate initial values

NParticles = 5000 # 5000 used for production runs
#np.random.seed(1)
#stz_inits = np.random.uniform(size=(NParticles, 3))
#vpar_inits = vpar0*np.random.uniform(size=(NParticles, 1))

stz_inits = np.column_stack((s0_vals, theta0_vals, zeta0_vals))
vpar_inits = vpar0_vals.reshape(-1, 1)

smin = 0.2
smax = 0.6
thetamin = 0
thetamax = np.pi
zetamin = 0
zetamax = np.pi
#stz_inits[:, 0] = stz_inits[:, 0]*(smax-smin) + smin
#stz_inits[:, 1] = stz_inits[:, 1]*(thetamax-thetamin) + thetamin
#stz_inits[:, 2] = stz_inits[:, 2]*(zetamax-zetamin) + zetamin

## using the default adaptive RK45 solver
solver_options = {'abstol': abstol, 'reltol': reltol}

t = time.time()



[Pauls-Laptop.local:14572] shmem: mmap: an error occurred while determining whether or not /var/folders/j4/72jv7vms357_6ywzplrwhkvh0000gn/T//ompi.Pauls-Laptop.501/jf.0/2824994816/sm_segment.Pauls-Laptop.501.a8620000.0 could be created.


In [20]:
## Call tracing routine
gc_tys, res_hits = trace_particles_boozer(
        field, stz_inits, -1 * vpar_inits, tmax=tmax, mass=mass, charge=charge, comm=comm,
        Ekin=Ekin, stopping_criteria=[MinToroidalFluxStoppingCriterion(1e-5),MaxToroidalFluxStoppingCriterion(1.0)],
        forget_exact_path=False, mode='gc_noK',solver_options=solver_options, dt_save=1e-3/999)

In [6]:
print(gc_tys)
print(np.shape(gc_tys[0]))

[array([[ 0.00000000e+00,  5.22276965e-01,  5.59852742e+00,
         4.87096066e-01, -2.57231284e+06],
       [ 1.00100100e-06,  5.33597230e-01,  5.49228785e+00,
         2.69053513e-01, -2.42215260e+06],
       [ 2.00200200e-06,  5.46339972e-01,  5.39278181e+00,
         6.38792624e-02, -2.25502856e+06],
       ...,
       [ 9.98998999e-04,  5.24234209e-01,  5.57032087e+00,
         2.72739121e+01, -2.53265634e+06],
       [ 1.00000000e-03,  5.35948716e-01,  5.46585252e+00,
         2.70591950e+01, -2.38001403e+06],
       [ 1.00000000e-03,  5.35948716e-01,  5.46585252e+00,
         2.70591950e+01, -2.38001403e+06]]), array([[0.00000000e+00, 3.19042985e-02, 4.05388573e+00, 3.99506655e+00,
        3.18097346e+06],
       [1.00100100e-06, 3.58665664e-02, 4.21921031e+00, 4.29649972e+00,
        3.23486808e+06],
       [2.00200200e-06, 4.04808430e-02, 4.37367318e+00, 4.60261056e+00,
        3.29600084e+06],
       ...,
       [9.98998999e-04, 6.49698340e-02, 1.45862579e+02, 3.37699754e+02

In [21]:
if (comm.rank == 0):
   res_lost = []
   loss_ctr = 0
   for i in range(len(res_hits)):
      if (len(res_hits[i])!=0):
         loss_ctr += 1
         res_lost.append(res_hits[i][0,:])
   print(f'Particles lost {loss_ctr}/{NParticles}={(100*loss_ctr)//NParticles:d}%')
   # np.savetxt('res_hits.txt',res_lost)
   np.savetxt('./trajectories/path_simsopt.txt', gc_tys[0])


Particles lost 1/5000=0%
